In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
import re


In [ ]:
data = pd.read_csv('/kaggle/input/us-crime-data/US_Crime_Data.csv')
data.sample(10)

In [ ]:
data.isna().sum()

## For this model we just need headlines

In [ ]:
df = data[['Title']]
df = df.dropna().reset_index(drop = True)
df.head()

In [ ]:
df['Title'][123]

In [ ]:
df.shape

# Removing Punctuations

In [ ]:
def clean_text(df):
    df['Title'] = df['Title'].apply(lambda x : x.lower())
    tokens = df['Title'].str.replace('[{}]'.format(string.punctuation), '')
    return tokens

In [ ]:
tokens = clean_text(df)

In [ ]:
tokens

In [ ]:
len(set(tokens))

# Tokenizing

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(tokens)
seq = tokenizer.texts_to_sequences(tokens)

In [ ]:
seq[:10]

# Creating input and output data list

In [ ]:
x = []
y = []
total_words_drop = 0
for i in seq:
    if len(i) > 1:
        for j in range(1, len(i)):
            x.append(i[:j])
            y.append(i[j])
            
    else : 
        total_words_drop +=1
print('Total Words Dropped : {}'.format(total_words_drop))

In [ ]:
y[: 10]

# Padding sequences

In [ ]:
x = tf.keras.preprocessing.sequence.pad_sequences(x)

In [ ]:
x.shape

# Shaping y same as x

In [ ]:
y = tf.keras.utils.to_categorical(y)

In [ ]:
y.shape

# Vocab Size : total no. of unique words

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

# LSTM Model

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,49 ),
                            tf.keras.layers.LSTM(100, return_sequences = True),
                            tf.keras.layers.LSTM(100),
                            tf.keras.layers.Dense(100, activation = 'relu'),
                            tf.keras.layers.Dense(vocab_size, activation = 'softmax')])

In [ ]:
model.summary()

In [ ]:
model.compile(loss  = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'],
             )

In [ ]:
history = model.fit(x,y,
                   epochs = 100,
                    batch_size = 256,
                    callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                               patience = 5,
                                                               restore_best_weights = True))

# Saving model

In [ ]:
model.save('model.h5')

# Vocab Array : list of all the unique words

In [ ]:
vocab_array = np.array(list(tokenizer.word_index.keys()))
vocab_array

# Final Function for Predictions

In [ ]:
def make_predictions(text, n_words):
    for i in range(n_words):
        text_tokenize = tokenizer.texts_to_sequences([text])
        text_padding  = tf.keras.preprocessing.sequence.pad_sequences(text_tokenize, maxlen = 49)
        prediction = np.squeeze(np.argmax(model.predict(text_padding), axis = -1))
        prediction = str(vocab_array[prediction - 1])
        text += " " + prediction
    return text

# Testing Model

In [ ]:
make_predictions('california',5)

In [ ]:
make_predictions('new york',8)

In [ ]:
make_predictions('highway',8)

> Its so fun to get prediction !

# Thanks!